In [21]:
import streamlit as st
import pandas as pd
import requests
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# Collecte des données

In [22]:
import requests
import pandas as pd

def fetch_models_data():
    response = requests.get(
        "https://huggingface.co/api/models",
        params={"limit": 10000, "full": "True", "config": "True"}
    )
    if response.status_code == 200:
        data = response.json()
        models_data = []
        for model in data:
            models_data.append({
                "ID": model.get("id"),
                "Auteur": model.get("author"),
                "Gated": model.get("gated"),
                "Inference": model.get("inference"),
                "Dernière modification": model.get("lastModified"),
                "Likes": model.get("likes"),
                "Trending Score": model.get("trendingScore"),
                "Privé": model.get("private"),
                "Téléchargements": model.get("downloads"),
                "Tags": model.get("tags"),
                "Library": model.get("library_name"),
                "Date de création": model.get("createdAt"),
            })
        return pd.DataFrame(models_data)
    else:
        raise Exception(f"Erreur de chargement des données ({response.status_code})")

# Exemple d'utilisation
df = fetch_models_data()
df.head()

,ID,Auteur,Gated,Inference,Dernière modification,Likes,Trending Score,Privé,Téléchargements,Tags,Library,Date de création
0,Qwen/Qwen2.5-Coder-32B-Instruct,Qwen,False,warm,2024-11-18T12:55:03.000Z,956,309.0,False,68370,"[transformers, safetensors, qwen2, text-genera...",transformers,2024-11-06T07:49:50.000Z
1,mistralai/Pixtral-Large-Instruct-2411,mistralai,auto,explicit-opt-out,2024-11-19T19:06:25.000Z,297,297.0,False,382,"[vllm, en, fr, de, es, it, pt, zh, ja, ru, ko,...",vllm,2024-11-14T20:07:26.000Z
2,NexaAIDev/omnivision-968M,NexaAIDev,False,library-not-detected,2024-11-24T01:05:25.000Z,387,270.0,False,7755,"[gguf, multimodal, conversational, GGUF, Image...",None,2024-11-14T01:42:29.000Z
3,AIDC-AI/Marco-o1,AIDC-AI,False,explicit-opt-out,2024-11-23T10:40:27.000Z,238,237.0,False,1083,"[transformers, safetensors, qwen2, text-genera...",transformers,2024-11-13T02:37:28.000Z
4,black-forest-labs/FLUX.1-dev,black-forest-labs,auto,warm,2024-08-16T14:38:19.000Z,6622,213.0,False,1284866,"[diffusers, safetensors, text-to-image, image-...",diffusers,2024-07-31T21:13:44.000Z


# Exploration des données 

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ID                     1000 non-null   object 
 1   Auteur                 1000 non-null   object 
 2   Gated                  1000 non-null   object 
 3   Inference              1000 non-null   object 
 4   Dernière modification  1000 non-null   object 
 5   Likes                  1000 non-null   int64  
 6   Trending Score         1000 non-null   float64
 7   Privé                  1000 non-null   bool   
 8   Téléchargements        1000 non-null   int64  
 9   Tags                   1000 non-null   object 
 10  Library                805 non-null    object 
 11  Date de création       1000 non-null   object 
dtypes: bool(1), float64(1), int64(2), object(8)
memory usage: 87.0+ KB


In [25]:
df.describe()

,Likes,Trending Score,Téléchargements
count,1000.000000,1000.000000,1.000000e+03
mean,327.632000,10.051000,1.281296e+06
std,710.252546,24.526203,1.407676e+07
min,3.000000,2.000000,0.000000e+00
25%,16.000000,3.000000,3.352500e+02
50%,78.000000,4.000000,7.017000e+03
75%,314.500000,8.000000,1.069845e+05
max,6622.000000,309.000000,4.148695e+08


In [26]:
# Filtrer uniquement les colonnes numériques
numerical_columns = df.select_dtypes(include="number").columns

# Création d'un menu déroulant pour sélectionner la colonne
dropdown = widgets.Dropdown(
    options=numerical_columns,
    description='Variable :',
    style={'description_width': 'initial'}
)

# Fonction pour tracer un histogramme et afficher un seul graphique
def plot_histogram(change):
    clear_output(wait=True)  # Effacer le graphique précédent
    display(dropdown)  # Réafficher le menu déroulant
    selected_column = dropdown.value
    if selected_column:  # Vérifier qu'une colonne est sélectionnée
        fig = px.histogram(
            df,
            x=selected_column,
            nbins=50,  # Ajuster le nombre de bins si nécessaire
            title=f"Distribution de {selected_column}",
            labels={selected_column: selected_column},
        )
        fig.update_layout(
            xaxis_title=selected_column,
            yaxis_title="Fréquence",
            template="plotly_white",
        )
        fig.show()

# Associer la fonction au changement de valeur du menu déroulant
dropdown.observe(plot_histogram, names='value')

# Afficher le menu déroulant
display(dropdown)

Dropdown(description='Variable :', options=('Likes', 'Trending Score', 'Téléchargements'), style=DescriptionSt…

In [27]:
# Convertir la colonne "Date de création" en datetime
df['Date de création'] = pd.to_datetime(df['Date de création'], errors='coerce')

# Supprimer les valeurs nulles (si des dates sont mal formatées ou manquantes)
df = df.dropna(subset=['Date de création'])

# Grouper les données par mois pour compter le nombre d'articles
articles_per_time = df.groupby(df['Date de création'].dt.to_period("M")).size().reset_index(name='Nombre d\'articles')

# Convertir les périodes en datetime pour une meilleure manipulation
articles_per_time['Date de création'] = articles_per_time['Date de création'].dt.to_timestamp()

# Tracer la série chronologique avec Plotly
fig = px.line(
    articles_per_time,
    x='Date de création',
    y='Nombre d\'articles',
    title="Nombre d'articles au cours du temps",
    labels={
        'Date de création': 'Date',
        'Nombre d\'articles': 'Nombre d\'articles'
    },
)

# Mettre à jour le style du graphique
fig.update_layout(
    template="plotly_white",
    xaxis_title="Date",
    yaxis_title="Nombre d'articles",
)

# Afficher le graphique
fig.show()

/var/folders/tb/ydg53kwn3bs7hz_m0vxtdn200000gn/T/ipykernel_21468/1396546602.py:8: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



# Prétraitement des données

In [28]:
# Pré-traitements de données
def preprocess_data(df):
    # Conversion des colonnes numériques
    df['Likes'] = pd.to_numeric(df['Likes'], errors='coerce')
    df['Téléchargements'] = pd.to_numeric(df['Téléchargements'], errors='coerce')
    
    # Conversion des dates
    df['Date de création'] = pd.to_datetime(df['Date de création'], errors='coerce')
    
    # Création d'une colonne pour regrouper les modèles par mois
    df['Mois'] = pd.to_datetime(df['Date de création']).dt.to_period('M')
    
    return df

df = preprocess_data(df)
df.head()

/var/folders/tb/ydg53kwn3bs7hz_m0vxtdn200000gn/T/ipykernel_21468/2108194935.py:11: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



,ID,Auteur,Gated,Inference,Dernière modification,Likes,Trending Score,Privé,Téléchargements,Tags,Library,Date de création,Mois
0,Qwen/Qwen2.5-Coder-32B-Instruct,Qwen,False,warm,2024-11-18T12:55:03.000Z,956,309.0,False,68370,"[transformers, safetensors, qwen2, text-genera...",transformers,2024-11-06 07:49:50+00:00,2024-11
1,mistralai/Pixtral-Large-Instruct-2411,mistralai,auto,explicit-opt-out,2024-11-19T19:06:25.000Z,297,297.0,False,382,"[vllm, en, fr, de, es, it, pt, zh, ja, ru, ko,...",vllm,2024-11-14 20:07:26+00:00,2024-11
2,NexaAIDev/omnivision-968M,NexaAIDev,False,library-not-detected,2024-11-24T01:05:25.000Z,387,270.0,False,7755,"[gguf, multimodal, conversational, GGUF, Image...",None,2024-11-14 01:42:29+00:00,2024-11
3,AIDC-AI/Marco-o1,AIDC-AI,False,explicit-opt-out,2024-11-23T10:40:27.000Z,238,237.0,False,1083,"[transformers, safetensors, qwen2, text-genera...",transformers,2024-11-13 02:37:28+00:00,2024-11
4,black-forest-labs/FLUX.1-dev,black-forest-labs,auto,warm,2024-08-16T14:38:19.000Z,6622,213.0,False,1284866,"[diffusers, safetensors, text-to-image, image-...",diffusers,2024-07-31 21:13:44+00:00,2024-07


# Modèle le Plus Populaire par Mois

In [29]:
# Préparer les données
timeline_df = df.copy()
timeline_df['Mois'] = pd.to_datetime(timeline_df['Date de création']).dt.to_period('M')

# Trouver le top modèle par mois
top_monthly = timeline_df.sort_values('Likes', ascending=False).groupby('Mois').first().reset_index()
top_monthly['Mois'] = top_monthly['Mois'].astype(str)

# Créer la visualisation
fig = go.Figure()

# Ajouter les points pour les top modèles
fig.add_trace(go.Scatter(
    x=pd.to_datetime(top_monthly['Mois']),
    y=top_monthly['Likes'],
    mode='markers+text',
    marker=dict(
        color='#FFD700',
        size=15,
        symbol='star',
    ),
    text=top_monthly['ID'].apply(lambda x: x.split('/')[-1]),  # Juste le nom du modèle
    textposition="top center",
    hovertemplate="<b>%{text}</b><br>" +
                  "Likes: %{y}<br>" +
                  "Date: %{x|%B %Y}<br>" +
                  "<extra></extra>"
))

# Ajouter des lignes entre les points
fig.add_trace(go.Scatter(
    x=pd.to_datetime(top_monthly['Mois']),
    y=top_monthly['Likes'],
    mode='lines',
    line=dict(color='#FFD700', width=1, dash='dot'),
    hoverinfo='skip'
))

# Mise en page
fig.update_layout(
    title="Modèle le Plus Liké Chaque Mois",
    xaxis_title="Date",
    yaxis_title="Nombre de Likes",
    template="plotly_dark",
    showlegend=False,
    height=400,
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
    yaxis=dict(gridcolor='rgba(128,128,128,0.2)'),
    xaxis=dict(gridcolor='rgba(128,128,128,0.2)'),
    hoverlabel=dict(
        bgcolor="black",
        font_size=12
    )
)

# Afficher le graphique
fig.show()

/var/folders/tb/ydg53kwn3bs7hz_m0vxtdn200000gn/T/ipykernel_21468/2185550661.py:3: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



# Camembert de Tags

In [30]:
# Extraire les tags
tags_series = pd.Series([tag for sublist in df['Tags'].dropna() for tag in sublist])

# Calculer la fréquence des tags
if not tags_series.empty:
    tag_counts = tags_series.value_counts().reset_index(name="count")
    
    # Visualisation en pie chart
    fig_tags = px.pie(
        tag_counts.head(10),
        names="index",
        values="count",
        title="Top 10 Tags",
        color_discrete_sequence=px.colors.sequential.Sunset
    )
    fig_tags.update_traces(textinfo='percent+label')
    
    # Afficher le graphique
    fig_tags.show()
else:
    print("Aucun tag disponible.")